In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
import cv2
import torch
import torch
import requests
import PIL
from PIL import Image, ImageDraw, ImageFont
from PIL import Image
from PIL import Image
import requests
import io
import math
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
!pip install git+https://github.com/cocodataset/panopticapi.git

  Cloning https://github.com/cocodataset/panopticapi.git to /tmp/pip-req-build-664a6a19
  Running command git clone -q https://github.com/cocodataset/panopticapi.git /tmp/pip-req-build-664a6a19


In [ ]:
import panopticapi
from panopticapi.utils import id2rgb, rgb2id
from panopticapi.utils import rgb2id, id2rgb

In [ ]:
import torchvision.transforms as T

# standard PyTorch mean-std input image normalization
transform = T.Compose([
    T.Resize((800,600)),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# for output bounding box post-processing
def box_cxcywh_to_xyxy(x):
    x_c, y_c, w, h = x.unbind(1)
    b = [(x_c - 0.5 * w), (y_c - 0.5 * h),
         (x_c + 0.5 * w), (y_c + 0.5 * h)]
    return torch.stack(b, dim=1)

def rescale_bboxes(out_bbox, size):
    img_w, img_h = size
    b = box_cxcywh_to_xyxy(out_bbox)
    b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32)
    return b

In [ ]:
def filter_bboxes_from_outputs(outputs,
                               threshold=0.7):
  
  # keep only predictions with confidence above threshold
  probas = outputs['pred_logits'].softmax(-1)[0, :, :-1]
  keep = probas.max(-1).values > threshold

  probas_to_keep = probas[keep]

  # convert boxes from [0; 1] to image scales
  bboxes_scaled = rescale_bboxes(outputs['pred_boxes'][0, keep], im.size)
  
  return probas_to_keep, bboxes_scaled

In [ ]:
import itertools
import seaborn as sns
palette = itertools.cycle(sns.color_palette())



In [ ]:
def plot_finetuned_results(pil_img, seg_img, prob=None, boxes=None):
    #plt.figure(figsize=(8,8))
    plt.subplot(1, 4, 3)
    plt.imshow(pil_img)
    #plt.axis('off')
    
    ax = plt.gca()
    colors = COLORS * 100
    if prob is not None and boxes is not None:
      for p, (xmin, ymin, xmax, ymax), c in zip(prob, boxes.tolist(), colors):
          ax.add_patch(plt.Rectangle((xmin, ymin), xmax - xmin, ymax - ymin,
                                    fill=False, color=c, linewidth=3))
          cl = p.argmax()
          text = f'{finetuned_classes[cl]}: {p[cl]:0.2f}'
          ax.text(xmin, ymin, text, fontsize=15,
                  bbox=dict(facecolor='yellow', alpha=0.5))
    plt.axis('off')
    #plt.figure(figsize=(10,3))
    #plt.show()
    

    plt.subplot(1, 4, 4)
    #plt.figure(figsize=(5,4))
    #seg_img = seg_img.resize((256, 256))
    plt.imshow(seg_img)
    plt.axis('off')
    plt.show()

In [ ]:
def run_worflow(my_image, my_model):
  # mean-std normalize the input image (batch-size: 1)
  img = transform(my_image).unsqueeze(0)

  # propagate through the model
  outputs = my_model(img)
  palette = itertools.cycle(sns.color_palette())
  result = postprocessor(outputs, torch.as_tensor(img.shape[-2:]).unsqueeze(0))[0]
  # The segmentation is stored in a special-format png
  panoptic_seg = Image.open(io.BytesIO(result['png_string']))
  panoptic_seg = np.array(panoptic_seg, dtype=np.uint8).copy()
  # We retrieve the ids corresponding to each mask
  panoptic_seg_id = rgb2id(panoptic_seg)

  # Finally we color each mask individually
  panoptic_seg[:, :, :] = 0
  for id in range(panoptic_seg_id.max() + 1):
    panoptic_seg[panoptic_seg_id == id] = np.asarray(next(palette)) * 255

  for threshold in [0.91]:
    
    probas_to_keep, bboxes_scaled = filter_bboxes_from_outputs(outputs,
                                                              threshold=threshold)

    
    
    plot_finetuned_results(my_image,
                           panoptic_seg,
                           probas_to_keep, 
                           bboxes_scaled)

In [ ]:
def filter_bboxes_from_outputs(outputs,
                               threshold=0.7):
  
  # keep only predictions with confidence above threshold
  probas = outputs['pred_logits'].softmax(-1)[0, :, :-1]
  keep = probas.max(-1).values > threshold

  probas_to_keep = probas[keep]

  # convert boxes from [0; 1] to image scales
  bboxes_scaled = rescale_bboxes(outputs['pred_boxes'][0, keep], im.size)
  
  return probas_to_keep, bboxes_scaled

In [ ]:
finetuned_classes =['rebar','spall','crack','stuff']

COLORS = [[0.000, 0.447, 0.741], [0.850, 0.325, 0.098], [0.929, 0.694, 0.125],
          [0.494, 0.184, 0.556], [0.466, 0.674, 0.188], [0.301, 0.745, 0.933]]
colors = COLORS *2     

In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/EVA7 Capstone/Part 3/

/content/drive/MyDrive/Colab Notebooks/EVA7 Capstone/Part 3


In [ ]:
import torch
import torch.nn.functional as F
from torch import nn


In [ ]:
#PATH = '/content/drive/MyDrive/Colab Notebooks/EVA7 Capstone/Part 3/detr/part2_output/checkpoint.pth'
PATH = '/content/drive/MyDrive/Colab Notebooks/EVA7 Capstone/Part 3/detr/segmentation_output/checkpoint.pth'
model,postprocessor = torch.hub.load('facebookresearch/detr', 'detr_resnet50_panoptic', pretrained=False,return_postprocessor=True,num_classes=4)

#from detr import (build)

checkpoint = torch.load(PATH,map_location='cpu',)
#/content/drive/MyDrive/Colab Notebooks/Session_14/detr/outputs
#/content/drive/MyDrive/Colab Notebooks/EVA7 Capstone/Part 3/detr/output_2
model.load_state_dict(checkpoint['model'],strict=True)
my_args = vars(checkpoint['args'])
my_args['device'] = 'cpu'
#my_args = argparse.Namespace(**my_args)
#build_op = detr.build(my_args)
model.eval();

Using cache found in /root/.cache/torch/hub/facebookresearch_detr_main


In [ ]:
from PIL import Image

from os import listdir
counter = 0

imagesList = listdir('/content/drive/MyDrive/Colab Notebooks/EVA7 Capstone/Part 3/detr/coco_panoptic/panoptic_train2017/')
for images in imagesList:
  #img_name = '/content/drive/MyDrive/Colab Notebooks/Session_14/data/custom/train2017/14666848163_8be8e37562_k.jpg'
  #img_name = '/content/drive/MyDrive/Colab Notebooks/Session_14/data/custom/Test_baloon.jpg'
  im = Image.open(f'/content/drive/MyDrive/Colab Notebooks/EVA7 Capstone/Part 3/detr/coco/images/{images[:-4]}.jpg') #30-5.jpg ,0103, 0012,12-9,00026,001134,
  try:
    gt = Image.open(f'/content/drive/MyDrive/Colab Notebooks/EVA7 Capstone/Part 3/detr/coco_panoptic/panoptic_train2017/{images}')
  except:
    gt = Image.open(f'/content/drive/MyDrive/Colab Notebooks/EVA7 Capstone/Part 3/detr/coco_panoptic/panoptic_val2017/{images[:-4]}.png')
  #000136,12-9,0012,0036901
  # Difficult img  -- 0036901
  #im = Image.open(img_name)
  plt.figure(figsize=(25,25))
  plt.subplot(1, 4, 1)
  plt.imshow(im)
  plt.axis('off')
  plt.subplot(1, 4, 2)
  plt.imshow(gt)
  plt.axis('off')
  
  run_worflow(im,model)
  
  
  plt.show()
  img = transform(im).unsqueeze(0)
  out = model(img)
  if counter > 100:
    break
  counter = counter + 1


In [ ]:
pause

NameError: ignored

In [ ]:
cd /content/drive/MyDrive/Colab Notebooks/EVA7 Capstone/Part 3/detr/models

In [ ]:
type(postprocessor)

In [ ]:
import models.segmentation

In [ ]:
result = postprocessor(out, torch.as_tensor(img.shape[-2:]).unsqueeze(0))[0]
result['segments_info']

In [ ]:
type(out['pred_logits']),out['pred_logits'].shape
#out['pred_logits'].reshape(100,5)

In [ ]:
is_thing_map = {i: i <= 2 for i in range(5)}
is_thing_map

In [ ]:
stuff_equiv_classes = defaultdict(lambda: [])

In [ ]:

scores, labels = out['pred_logits'].softmax(-1).max(-1)
keep = labels.ne(out["pred_logits"].shape[-1] - 1) & (scores > .95)
cur_scores, cur_classes = out["pred_logits"].softmax(-1).max(-1)
cur_classes = cur_classes[keep]
for k, label in enumerate(cur_classes):
  print('loop')
  if not is_thing_map[label.item()]:
    print(is_thing_map[label.item()])
    print(label.item())
    stuff_equiv_classes[label.item()].append(k)

In [ ]:
labels

In [ ]:
stuff_equiv_classes.values()

In [ ]:
for equiv in stuff_equiv_classes.values():
  print(type(equiv))
  if len(equiv) > 1:
    print('found')

In [ ]:
import itertools
import seaborn as sns
palette = itertools.cycle(sns.color_palette())

# The segmentation is stored in a special-format png
panoptic_seg = Image.open(io.BytesIO(result['png_string']))
panoptic_seg = np.array(panoptic_seg, dtype=np.uint8).copy()
# We retrieve the ids corresponding to each mask
panoptic_seg_id = rgb2id(panoptic_seg)

# Finally we color each mask individually
panoptic_seg[:, :, :] = 0
for id in range(panoptic_seg_id.max() + 1):
  panoptic_seg[panoptic_seg_id == id] = np.asarray(next(palette)) * 255
plt.figure(figsize=(10,10))

plt.imshow(panoptic_seg)
plt.axis('off')
plt.show()

In [ ]:
import numpy as np
from PIL import Image
from panopticapi.utils import rgb2id
#ann_path = '/content/drive/MyDrive/Colab Notebooks/EVA7 Capstone/Part 3/detr/coco_panoptic/panoptic_train2017/0009203.png'
ann_path = '/content/drive/MyDrive/Colab Notebooks/EVA7 Capstone/dataset/masks/00026rebar.jpg'
msk_im = Image.open(ann_path)
masks = np.asarray(msk_im, dtype=np.uint32)
#masks[masks < 50] = 0 
#masks[masks > 50] = 255 
#masks = rgb2id(masks)

plt.imshow(masks)


In [ ]:
out.keys()

In [ ]:
from util.plot_utils import plot_logs

from pathlib import Path

log_directory = [Path('/content/drive/MyDrive/Colab Notebooks/EVA7 Capstone/Part 3/detr/segmentation_output')]

In [ ]:
fields_of_interest = (
    'loss',
    'mAP',
    )

plot_logs(log_directory,
          fields_of_interest)

In [ ]:
fields_of_interest = (
    'loss_ce',
    'loss_bbox',
    'loss_giou',
  
    
    )

plot_logs(log_directory,
          fields_of_interest)

In [ ]:
fields_of_interest = (
    'class_error',
    'cardinality_error_unscaled',
    )

plot_logs(log_directory,
          fields_of_interest) 

In [ ]:
from util.plot_utils import plot_logs

from pathlib import Path

bbox_log_directory = [Path('/content/drive/MyDrive/Colab Notebooks/EVA7 Capstone/Part 3/detr/bbox_output')]

In [ ]:
fields_of_interest = (
    'loss',
    'mAP',
    )

plot_logs(bbox_log_directory,
          fields_of_interest)

In [ ]:
fields_of_interest = (
    'loss_ce',
    'loss_bbox',
    'loss_giou',
  
    
    )

plot_logs(bbox_log_directory,
          fields_of_interest)

In [ ]:
fields_of_interest = (
    'class_error',
    'cardinality_error_unscaled',
    )

plot_logs(bbox_log_directory,
          fields_of_interest) 

In [ ]:
# compute the scores, excluding the "no-object" class (the last one)
scores = out["pred_logits"].softmax(-1)[..., :-1].max(-1)[0]
# threshold the confidence
keep = scores > 0.85

# Plot all the remaining masks
ncols = 5
fig, axs = plt.subplots(ncols=ncols, nrows=5, figsize=(18, 10)) #nrows=math.ceil(keep.sum().item() / ncols)
for line in axs:
    for a in line:
        a.axis('off')
for i, mask in enumerate(out["pred_masks"][keep]):
    ax = axs[i // ncols, i % ncols]
    ax.imshow(mask, cmap="cividis")
    ax.axis('off')
fig.tight_layout()

## Find invalid images Assertion Error

In [ ]:
def masks_to_boxes(masks):
    """Compute the bounding boxes around the provided masks

    The masks should be in format [N, H, W] where N is the number of masks, (H, W) are the spatial dimensions.

    Returns a [N, 4] tensors, with the boxes in xyxy format
    """
    if masks.numel() == 0:
        return torch.zeros((0, 4), device=masks.device)
        print('Error Found')
    h, w = masks.shape[-2:]
    #print(f'H- {h}, W - {w}')
    y = torch.arange(0, h, dtype=torch.float)
    x = torch.arange(0, w, dtype=torch.float)
    y, x = torch.meshgrid(y, x)
    #print(f'x shape-{x.shape} y shape - {y.shape}')
    x_mask = (masks * x.unsqueeze(0))
    #print(x_mask.shape)
    x_max = x_mask.flatten(1).max(-1)[0]
    x_min = x_mask.masked_fill(~(masks.bool()), 1e8).flatten(1).min(-1)[0]

    y_mask = (masks * y.unsqueeze(0))
    y_max = y_mask.flatten(1).max(-1)[0]
    y_min = y_mask.masked_fill(~(masks.bool()), 1e8).flatten(1).min(-1)[0]

    return torch.stack([x_min, y_min, x_max, y_max], 1)

In [ ]:
import json
ann_file = '/content/drive/MyDrive/Colab Notebooks/EVA7 Capstone/Part 3/detr/coco_panoptic/annotations/panoptic_train2017.json'
with open(ann_file, 'r') as f:
  json_file = json.load(f)

In [ ]:

for img, ann in zip(json_file['images'], json_file['annotations']):
  #if ann['image_id'] == -img_id:
    #break
  ann_path = f'/content/drive/MyDrive/Colab Notebooks/EVA7 Capstone/Part 3/detr/coco_panoptic/panoptic_train2017/{img["file_name"][:-4]}.png'
  #img1 = Image.open(ann_path)
  masks = np.asarray(Image.open(ann_path), dtype=np.uint32)
  masks = rgb2id(masks)
  ids = np.array([ann['id'] for ann in ann['segments_info']])
  masks = masks == ids[:, None, None]
  masks = torch.as_tensor(masks, dtype=torch.uint8)
  x = masks_to_boxes(masks)
  assert (ann['file_name'][:-4] == img['file_name'][:-4])
  try:
    assert (x[:, 2:] >= x[:, :2]).all()
  except:
  #print(img['file_name'])
    print(f'\'{ann["file_name"][:-4]}\',')

#img['file_name']